In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjqtHzbpirMey5i-WYhytn3gkfyXGFZ8ktb-pDiv7ru4eAbZ1S4qg4
Mounted at /content/drive


# Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 222kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617633 sha256=26f32086895cd56c21c5f8334725a24cf1cc4847756c92804334deccc549a8c2
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD,NMF
from surprise import accuracy
from surprise.model_selection import train_test_split, GridSearchCV,cross_validate

# Loading Data

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Project/ratings.csv')
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['userID', 'ISBN', 'bookRating']], reader)

# NMF

In [ ]:
algo = NMF()
cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5,verbose=True)
pd.DataFrame(cv_results).mean()

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.5130  2.5242  2.5029  2.5471  2.5575  2.5289  0.0205  
MAE (testset)     2.1755  2.1793  2.1690  2.2083  2.2173  2.1899  0.0192  
Fit time          4.11    4.07    3.99    3.83    4.11    4.02    0.11    
Test time         0.12    0.08    0.09    0.08    0.08    0.09    0.02    


test_rmse    2.528926
test_mae     2.189889
fit_time     4.021414
test_time    0.091804
dtype: float64

# SVD

In [ ]:
algo = SVD()
cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5,verbose=True)
pd.DataFrame(cv_results).mean()

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5055  1.5191  1.4959  1.5184  1.4883  1.5054  0.0122  
MAE (testset)     1.1426  1.1494  1.1384  1.1587  1.1449  1.1468  0.0069  
Fit time          3.09    2.91    2.93    2.93    3.02    2.98    0.07    
Test time         0.22    0.11    0.24    0.10    0.10    0.15    0.06    


test_rmse    1.505447
test_mae     1.146808
fit_time     2.976037
test_time    0.150671
dtype: float64

 It's clear that for the given dataset much better results can be obtained with SVD approach - both in terms of accuracy and fit time.

# Checking for best parameters for SVD

In [ ]:
param_grid = {'n_factors': [80,100,120],
              'n_epochs': [5, 10, 20],
             }

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


1.5160621334357882
{'n_factors': 80, 'n_epochs': 20}


# Splitting into train and test data

In [ ]:
trainset, testset = train_test_split(data, test_size=0.15,random_state=2)

# Training and testing model

In [ ]:
model = SVD(n_factors=80, n_epochs=20)
model.fit(trainset)

In [ ]:
test_pred = model.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 1.4826


1.482594584771045

# Results

In [ ]:
test_pred[0]

Prediction(uid=7346, iid='0449212750', r_ui=7.0, est=7.480700375092097, details={'was_impossible': False})

# Saving the model

In [ ]:
pickle.dump(model, open('/content/drive/MyDrive/Project/svd.sav', 'wb'))